# Exterior Dirichlet sphere problem

$$
\begin{equation}
\left\{ \begin{aligned} 
  -\Delta u = 0&,\;\;\Omega^+\\
  u = 1&,\;\; \Gamma 
\end{aligned} \right.
\end{equation}
$$

## Solve approach

In [ ]:
import bempp.api
import numpy as np

@bempp.api.real_callable
def one_fun(x, n, domain_index, res):
    res[0] = 1
@bempp.api.real_callable
def dirichlet_data(x, n, domain_index, result):
    result[0] = 1./(x[0]**2 + x[1]**2 + x[2]**2)**(0.5)


def execute(h):
    grid = bempp.api.shapes.sphere(h=h)

    dp0_space = bempp.api.function_space(grid, "P", 1)
    p1_space = bempp.api.function_space(grid, "P", 1)

    identity = bempp.api.operators.boundary.sparse.identity(
        p1_space, p1_space, dp0_space)
    dlp = bempp.api.operators.boundary.laplace.double_layer(
        p1_space, p1_space, dp0_space)
    slp = bempp.api.operators.boundary.laplace.single_layer(
        dp0_space, p1_space, dp0_space)
    # W = bempp.api.operators.boundary.laplace.hypersingular(p1_space, dp0_space, p1_space)
        
    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)
    one = bempp.api.GridFunction(dp0_space, fun=one_fun)
    rhs = (0.5 * identity - dlp)

    sol1, info, it1 = bempp.api.linalg.gmres(slp, rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    
    sol2, info, it2 = bempp.api.linalg.gmres(slp, rhs*dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    return ((one - sol1).l2_norm()) , it1, ((one - sol2).l2_norm()) , it2
    # print((one - sol).integrate()[0])
    # print("Number of iterations: {0}".format(it))
    # sol, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    # print((one - sol).integrate()[0])
    # print("Number of iterations: {0}".format(it))
    # sol, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    # print((one - sol).integrate()[0])
    # print("Number of iterations: {0}".format(it))

In [ ]:
H = [1, 0.5, 0.3, 0.1, 0.09, 0.07, 0.05]
L2_p1p1 = []
it_p1p1 = []
L2_p1p1_sf = []
it_p1p1_sf = []
for h in H:
    print(h)
    l21, it1, l22, it2 = execute(h)
    L2_p1p1.append(l21)
    it_p1p1.append(it1)
    L2_p1p1_sf.append(l22)
    it_p1p1_sf.append(it2)


In [ ]:
import scienceplots
import matplotlib.pyplot as plt

h_values = np.array(H)
h2 = h_values*h_values
h1 = h_values
h12 = (h_values**1.5)
h2 /= max(h2)
h1 /= max(h1)
h12 /= max(h12)

with plt.style.context(['science', 'high-vis']):
    plt.figure(figsize = (8,8))
    plt.plot(h1, L2_p1p1/max(L2_p1p1), "r^-", label="Base")
    plt.plot(h1, L2_p1p1_sf/max(L2_p1p1_sf), "bo-", label="Mass matrix")
    # plt.plot(h1, L2_p1p1_w/max(L2_p1p1_w), "gx-", label="Opposite order")
    # plt.plot(h1, L2_p1p1_wsf/max(L2_p1p1_wsf), "ys-", label="Both")

    plt.plot(h1, h1, "k--")
    plt.plot(h1,h2, "k--")
    h1 = h_values[:-1]
    # plt.plot(h1, L2_D0p1/max(L2_D0p1), "go-", label="$(DUAL0 / P_1)$")
    # plt.plot(h1, L2_D0p1_sf/max(L2_D0p1_sf), "g^-", label="$(DUAL0 / P_1)$")
    plt.xlabel("$h$")
    plt.ylabel("$L^2$ error")
    plt.xscale("log")
    plt.yscale("log")
    plt.axis("equal")
    plt.legend()
    # plt.xlim(plt.xlim()[::-1])


    plt.title("Dirichlet convergence of an exterior problem - $(DUAL_1 / DP_0)$")

    plt.show()

#### This cell uses the exafmm assembler - if not installed do not run

In [ ]:
import bempp.api
import numpy as np


def execute(h):
    grid = bempp.api.shapes.sphere(h=h)

    @bempp.api.real_callable
    def one_fun(x, n, domain_index, res):
        res[0] = 1
    @bempp.api.real_callable
    def dirichlet_data(x, n, domain_index, result):
        result[0] = 1.

    dp0_space = bempp.api.function_space(grid, "DUAL", 0)
    p1_space = bempp.api.function_space(grid, "P", 1)

    identity = bempp.api.operators.boundary.sparse.identity(
        p1_space, p1_space, dp0_space)
    dlp = bempp.api.operators.boundary.laplace.double_layer(
        p1_space, p1_space, dp0_space, assembler='fmm')
    slp = bempp.api.operators.boundary.laplace.single_layer(
        dp0_space, p1_space, dp0_space, assembler='fmm')
    # W = bempp.api.operators.boundary.laplace.hypersingular(p1_space, dp0_space, p1_space)
        
    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)
    one = bempp.api.GridFunction(dp0_space, fun=one_fun)
    rhs = (0.5 * identity - dlp)

    sol1, info, it1 = bempp.api.linalg.gmres(slp, rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    
    sol2, info, it2 = bempp.api.linalg.gmres(slp, rhs*dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    return ((one - sol1).l2_norm()) , it1, ((one - sol2).l2_norm()) , it2
    # print((one - sol).integrate()[0])
    # print("Number of iterations: {0}".format(it))
    # sol, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    # print((one - sol).integrate()[0])
    # print("Number of iterations: {0}".format(it))
    # sol, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    # print((one - sol).integrate()[0])
    # print("Number of iterations: {0}".format(it))

In [ ]:
H = [1, 0.5, 0.3, 0.1] ## Due to increased memory overhead from fmm assemble, using less mesh refinement
L2_D0p1 = []
it_D0p1 = []
L2_D0p1_sf = []
it_D0p1_sf = []
for h in H:
    print(h)
    l21, it1, l22, it2 = execute(h)
    L2_D0p1.append(l21)
    it_D0p1.append(it1)
    L2_D0p1_sf.append(l22)
    it_D0p1_sf.append(it2)

In [ ]:
import scienceplots
import matplotlib.pyplot as plt

H = [1, 0.5, 0.3, 0.1, 0.09, 0.07, 0.05]


h_values = np.array(H)
h2 = h_values*h_values
h1 = h_values
h12 = (h_values**1.5)
h2 /= max(h2)
h1 /= max(h1)
h12 /= max(h12)
with plt.style.context(['science', 'high-vis']):
    plt.figure(figsize = (8,8))
    plt.plot(h1, L2_p1p1/max(L2_p1p1), "y^-", label="$(P_1 / P_1)$")
    plt.plot(h1, L2_p1p1_sf/max(L2_p1p1_sf), "y^-", label="$(P_1 / P_1)$")

    plt.plot(h1, h1, "y--")
    plt.plot(h1, h12, "g--")
    plt.plot(h1,h2, "k--")
    h1 = h_values[:-3]
    plt.plot(h1, L2_D0p1/max(L2_D0p1), "go-", label="$(DUAL0 / P_1)$")
    plt.plot(h1, L2_D0p1_sf/max(L2_D0p1_sf), "g^-", label="$(DUAL0 / P_1)$")
    plt.xlabel("$h$")
    plt.ylabel("$L^2$ error")
    plt.xscale("log")
    plt.yscale("log")
    plt.axis("equal")
    plt.legend()
    # plt.xlim(plt.xlim()[::-1])


    plt.title("Dirichlet convergence of an interior mixed problem")

    plt.show()

In [ ]:
h_values = np.array(H)
h1 = h_values
with plt.style.context(['science']):
    plt.figure(figsize = (8,8))
    
    plt.plot(h1, it_p1p1, "ro-", label="$(P_1 / P_1)$")
    plt.plot(h1, it_p1p1_sf, "r^-", label="$(P_1 / P_1)$")
    h1 = h1[:-3]
    plt.plot(h1, it_D0p1, "yo-", label="$(DUAL_0 / P_1)$")
    plt.plot(h1, it_D0p1_sf, "y^-", label="$(DUAL_0 / P_1)$")
    plt.xlabel("$h$")
    plt.ylabel("GMRES iterations")
    plt.xscale("log")
    plt.yscale("log")
    plt.axis("equal")
    plt.legend()
    plt.xlim(plt.xlim()[::-1])
    plt.title("Iterations for solution of an Exterior Dirichlet problem")
    plt.show()

In [ ]:
# without dual - so without W - so use the spaces from the mixed case + dpo for u but only precond. with strong form (+conv)
# without dual - so with W - so use the spaces from the mixed case with both types of preconditioning (+conv)
# with dual - so with fmm - a variety of spaces but now with dual as well (+conv)
import bempp.api
import numpy as np
@bempp.api.real_callable
def one_fun(x, n, domain_index, res):
    res[0] = 1
@bempp.api.real_callable
def dirichlet_data(x, n, domain_index, result):
    result[0] = 1./(x[0]**2 + x[1]**2 + x[2]**2)**(0.5)
it_noprecond_p1p1 = []
it_sf_p1p1 = []
it_w_p1p1 = []
it_wsf_p1p1 = []
H = [1, 0.5, 0.3, 0.1, 0.09, 0.07, 0.05]
L2_noprecond_p1p1 = []
L2_sf_p1p1 = []
L2_w_p1p1 = []
L2_wsf_p1p1 = []
for h in H:

    grid = bempp.api.shapes.sphere(h=h)

    dp0_space = bempp.api.function_space(grid, "P", 1)
    p1_space = bempp.api.function_space(grid, "P", 1)

    identity = bempp.api.operators.boundary.sparse.identity(p1_space, p1_space, dp0_space)
    dlp = bempp.api.operators.boundary.laplace.double_layer(p1_space, p1_space, dp0_space)
    slp = bempp.api.operators.boundary.laplace.single_layer(dp0_space, p1_space, dp0_space)
    W = bempp.api.operators.boundary.laplace.hypersingular(p1_space, dp0_space, p1_space)

    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)
    one = bempp.api.GridFunction(dp0_space, fun=one_fun)

    rhs = (.5 * identity - dlp)

    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    it_noprecond_p1p1.append(it)
    L2_noprecond_p1p1.append((one - neumann_fun).l2_norm())
    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs*dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    it_sf_p1p1.append(it)
    L2_sf_p1p1.append((one - neumann_fun).l2_norm())
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    it_w_p1p1.append(it)
    L2_w_p1p1.append((one - neumann_fun).l2_norm())
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    it_wsf_p1p1.append(it)
    L2_wsf_p1p1.append((one - neumann_fun).l2_norm())

In [ ]:
import scienceplots
import matplotlib.pyplot as plt

with plt.style.context(['science']):
    plt.figure(figsize = (8,8))
    plt.plot(H, it_noprecond_p1p1, 'ro-',label='Base')
    plt.plot(H, it_sf_p1p1, 'bo-', label='Mass matrix')
    # plt.plot(H, it_w_p1p1, marker='go', label='Hypersingular')
    # plt.plot(H, it_wsf_p1p1, marker='yo', label='Both')
    plt.gca().set_xscale('log')
    plt.legend()
    plt.show()



In [ ]:
@bempp.api.real_callable
def one_fun(x, n, domain_index, res):
    res[0] = 1
@bempp.api.real_callable
def dirichlet_data(x, n, domain_index, result):
    result[0] = 1./(x[0]**2 + x[1]**2 + x[2]**2)**(0.5)
it_noprecond_fmm = []
it_sf_fmm = []
it_w_fmm = []
it_wsf_fmm = []
L2_noprecond_fmm = []
L2_sf_fmm = []
L2_w_fmm = []
L2_wsf_fmm = []
H = [1, 0.5, 0.3, 0.1]
for h in H:
    ## tables from pg53 scroggs
# h = 0.1
    grid = bempp.api.shapes.sphere(h=h)

    p1_space = bempp.api.function_space(grid, "P", 1)
    dual0_space = bempp.api.function_space(grid, "DUAL", 0)

    identity = bempp.api.operators.boundary.sparse.identity(
        p1_space, p1_space, dual0_space)
    dlp = bempp.api.operators.boundary.laplace.double_layer(
        p1_space, p1_space, dual0_space, assembler='fmm')
    slp = bempp.api.operators.boundary.laplace.single_layer(
        dual0_space, p1_space, dual0_space, assembler='fmm')
    W = bempp.api.operators.boundary.laplace.hypersingular(p1_space, dual0_space, p1_space, assembler='fmm')
        
    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)
    one = bempp.api.GridFunction(dual0_space, fun=one_fun)

    rhs = (.5 * identity- dlp)

    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    it_noprecond_fmm.append(it)
    
    L2_noprecond_fmm.append((one - neumann_fun).l2_norm())
    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs*dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    it_sf_fmm.append(it)
    L2_sf_fmm.append((one - neumann_fun).l2_norm())
    
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    it_w_fmm.append(it)
    L2_w_fmm.append((one - neumann_fun).l2_norm())
    
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    it_wsf_fmm.append(it)
    L2_wsf_fmm.append((one - neumann_fun).l2_norm())
    

In [ ]:
import scienceplots
import matplotlib.pyplot as plt
H = [1, 0.5, 0.3, 0.1, 0.09, 0.07, 0.05]
with plt.style.context(['science', 'high-vis']):
    plt.figure(figsize = (8,8))
    plt.plot(H, it_noprecond_p1p1, marker='o',label='Base')
    plt.plot(H, it_sf_p1p1, marker='o', label='Mass matrix')
    plt.plot(H, it_w_p1p1, marker='o', label='Hypersingular')
    plt.plot(H, it_wsf_p1p1, marker='o', label='Both')
    H = H[:-3]
    plt.plot(H, it_noprecond_fmm, marker='o',label='Base FMM')
    plt.plot(H, it_sf_fmm, marker='o', label='Mass matrix FMM')
    plt.plot(H, it_w_fmm, marker='o', label='Hypersingular FMM')
    plt.plot(H, it_wsf_fmm, marker='o', label='Both FMM')
    plt.gca().set_xscale('log')
    plt.xlim(plt.xlim()[::-1])
    
    plt.legend()
    plt.show()


## Internal Dirichlet example from Bempp.com augmented with preconditioning/new spaces/iteration analysis

In [ ]:
import bempp.api
import numpy as np

# without dual - so without W - so use the spaces from the mixed case + dpo for u but only precond. with strong form (+conv)
# without dual - so with W - so use the spaces from the mixed case with both types of preconditioning (+conv)
# with dual - so with fmm - a variety of spaces but now with dual as well (+conv)

i = []
ii = []
iii = []
iv = []
H = [1, 0.8, 0.5, 0.3, 0.1, 0.09, 0.07, 0.05]
for h in H:

    grid = bempp.api.shapes.sphere(h=h)

    dp0_space = bempp.api.function_space(grid, "P", 1)
    p1_space = bempp.api.function_space(grid, "P", 1)

    identity = bempp.api.operators.boundary.sparse.identity(
        p1_space, p1_space, dp0_space)
    dlp = bempp.api.operators.boundary.laplace.double_layer(
        p1_space, p1_space, dp0_space)
    slp = bempp.api.operators.boundary.laplace.single_layer(
        dp0_space, p1_space, dp0_space)
    W = bempp.api.operators.boundary.laplace.hypersingular(p1_space, dp0_space, p1_space)

    @bempp.api.real_callable
    def dirichlet_data(x, n, domain_index, result):
        result[0] = 1./(4 * np.pi * ((x[0] - .9)**2 + x[1]**2 + x[2]**2)**(0.5))
        # result[0] = 1./(4 * np.pi * ((x[0] - 1)**2 + x[1]**2 + x[2]**2)**(0.5))
        # result[0] = 1
        
    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)

    rhs = (.5 * identity + dlp)

    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    i.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs*dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    ii.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    iii.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    iv.append(it)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(H, i, marker='o')
plt.plot(H, ii, marker='o')
plt.plot(H, iii, marker='o')
plt.plot(H, iv, marker='o')
plt.gca().set_xscale('log')

In [ ]:
import bempp.api
import numpy as np


ic = []
iic = []
iiic = []
ivc = []
Hc = [1, 0.8, 0.5, 0.3, 0.1, 0.09, 0.07, 0.05]
for h in H:

    grid = bempp.api.shapes.cube(h=h)

    dp0_space = bempp.api.function_space(grid, "P", 1)
    p1_space = bempp.api.function_space(grid, "P", 1)

    identity = bempp.api.operators.boundary.sparse.identity(
        p1_space, p1_space, dp0_space)
    dlp = bempp.api.operators.boundary.laplace.double_layer(
        p1_space, p1_space, dp0_space)
    slp = bempp.api.operators.boundary.laplace.single_layer(
        dp0_space, p1_space, dp0_space)
    W = bempp.api.operators.boundary.laplace.hypersingular(p1_space, dp0_space, p1_space)

    @bempp.api.real_callable
    def dirichlet_data(x, n, domain_index, result):
        result[0] = 1./(4 * np.pi * ((x[0] - .9)**2 + x[1]**2 + x[2]**2)**(0.5))
        # result[0] = 1./(4 * np.pi * ((x[0] - 1)**2 + x[1]**2 + x[2]**2)**(0.5))
        # result[0] = 1
        
    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)

    rhs = (.5 * identity + dlp)

    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    ic.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs*dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    iic.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    iiic.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    ivc.append(it)

In [ ]:
plt.plot(H, i, marker='o')
plt.plot(H, ii, marker='o')
plt.plot(H, iii, marker='o')
plt.plot(H, iv, marker='o')
plt.plot(H, ic, marker='o')
plt.plot(H, iic, marker='o')
plt.plot(H, iiic, marker='o')
plt.plot(H, ivc, marker='o')
plt.gca().set_xscale('log')

In [ ]:
import bempp.api
import numpy as np


icc = []
iicc = []
iiicc = []
ivcc = []
H = [1, 0.8, 0.5, 0.3, 0.1]#, 0.09, 0.07, 0.05]
for h in H:
    ## tables from pg53 scroggs

    grid = bempp.api.shapes.cube(h=h)

    dp0_space = bempp.api.function_space(grid, "DUAL", 0)
    p1_space = bempp.api.function_space(grid, "P", 1)
    dual0_space = bempp.api.function_space(grid, "DUAL", 0)
    dual1_space = bempp.api.function_space(grid, "DUAL", 0)

    identity = bempp.api.operators.boundary.sparse.identity(
        p1_space, p1_space, dual0_space)
    dlp = bempp.api.operators.boundary.laplace.double_layer(
        p1_space, p1_space, dual0_space, assembler='fmm')
    slp = bempp.api.operators.boundary.laplace.single_layer(
        dual0_space, p1_space, dual0_space, assembler='fmm')
    W = bempp.api.operators.boundary.laplace.hypersingular(p1_space, dp0_space, p1_space, assembler='fmm')

    @bempp.api.real_callable
    def dirichlet_data(x, n, domain_index, result):
        result[0] = 1./(4 * np.pi * ((x[0] - .9)**2 + x[1]**2 + x[2]**2)**(0.5))
        # result[0] = 1./(4 * np.pi * ((x[0] - 1)**2 + x[1]**2 + x[2]**2)**(0.5))
        # result[0] = 1
        
    dirichlet_fun = bempp.api.GridFunction(p1_space, fun=dirichlet_data)

    rhs = (.5 * identity + dlp)

    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    icc.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(slp, rhs*dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    iicc.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, return_iteration_count=True)
    iiicc.append(it)
    neumann_fun, info, it = bempp.api.linalg.gmres(W*slp, W*rhs* dirichlet_fun, tol=1E-5, use_strong_form=True, return_iteration_count=True)
    ivcc.append(it)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(H, icc, marker='o')
plt.plot(H, iicc, marker='o')
plt.plot(H, iiicc, marker='o')
plt.plot(H, ivcc, marker='o')
plt.gca().set_xscale('log')

In [ ]:
grid = bempp.api.shapes.sphere(h=0.1)

const_space = bempp.api.function_space(grid, 'DP', 0)
lin_space = bempp.api.function_space(grid, 'P', 1)
bary_space = bempp.api.function_space(grid, 'DUAL', 0)
bary_lin_space = bempp.api.function_space(grid, 'DUAL', 1)

base_slp = bempp.api.operators.boundary.laplace.single_layer(bary_space, bary_lin_space, bary_space, assembler='fmm')
base_hyp = bempp.api.operators.boundary.laplace.hypersingular(bary_lin_space, bary_space, bary_lin_space, assembler='fmm')


@bempp.api.real_callable
def one_fun(x, n, domain_index, res):
    res[0] = 1

rhs = bempp.api.GridFunction(base_slp.range, fun=one_fun)
lhs = (base_slp*base_hyp).strong_form()

from scipy.sparse.linalg import gmres

phi_vec, _ = gmres(lhs, rhs.coefficients, tol=1e-5)
phi_fun = base_hyp*bempp.api.GridFunction(base_hyp.domain, coefficients=phi_vec)